<a href="https://www.kaggle.com/code/johnycoder/icr-inference?scriptVersionId=160907567" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
!pip install -q /kaggle/input/tabpfn-019-whl/tabpfn-0.1.9-py3-none-any.whl
!mkdir /opt/conda/lib/python3.10/site-packages/tabpfn/models_diff
!cp /kaggle/input/tabpfn-019-whl/prior_diff_real_checkpoint_n_0_epoch_100.cpkt /opt/conda/lib/python3.10/site-packages/tabpfn/models_diff/

In [2]:
import pandas as pd, numpy as np, catboost as ctb, pandas as pd, time, warnings
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import make_scorer, log_loss
from sklearn import svm, neighbors, naive_bayes, ensemble, neural_network, preprocessing, impute, gaussian_process, linear_model, compose, cluster
from sklearn.multioutput import MultiOutputClassifier
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline
from sklearn.ensemble import VotingClassifier

from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

from tabpfn import TabPFNClassifier
from joblib import dump, load

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [3]:
train = pd.read_csv("/kaggle/input/icr-identify-age-related-conditions/train.csv")
greeks = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/greeks.csv')
train = pd.merge(train, greeks, on='Id', how='inner')
classes = train.Class
train = train.drop(['Class', 'Id', 'Epsilon'], axis=1)
display(train)
features = train.columns
numeric_transformer = Pipeline(
    steps=[('imputer', impute.SimpleImputer(missing_values=pd.NA, strategy='mean')),
           ('scaler', preprocessing.MinMaxScaler())])

categoric_transformer = Pipeline(
    steps=[('imputer', impute.SimpleImputer(missing_values=pd.NA, strategy='most_frequent')),
           ('onehot', preprocessing.OneHotEncoder(handle_unknown='ignore'))])

preprocessor = compose.ColumnTransformer(transformers=[
        ("num", numeric_transformer, features.drop(['EJ', 'Alpha', 'Beta', 'Gamma', 'Delta'])),
        ("cat", categoric_transformer, ['EJ', 'Alpha', 'Beta', 'Gamma', 'Delta'])])

,AB,AF,AH,AM,AR,AX,AY,AZ,BC,BD,...,GB,GE,GF,GH,GI,GL,Alpha,Beta,Gamma,Delta
0,0.209377,3109.03329,85.200147,22.394407,8.138688,0.699861,0.025578,9.812214,5.555634,4126.58731,...,11.339138,72.611063,2003.810319,22.136229,69.834944,0.120343,B,C,G,D
1,0.145282,978.76416,85.200147,36.968889,8.138688,3.632190,0.025578,13.517790,1.229900,5496.92824,...,9.292698,72.611063,27981.562750,29.135430,32.131996,21.978000,A,C,M,B
2,0.470030,2635.10654,85.200147,32.360553,8.138688,6.732840,0.025578,12.824570,1.229900,5135.78024,...,37.077772,88.609437,13676.957810,28.022851,35.192676,0.196941,A,C,M,B
3,0.252107,3819.65177,120.201618,77.112203,8.138688,3.685344,0.025578,11.053708,1.229900,4169.67738,...,18.529584,82.416803,2094.262452,39.948656,90.493248,0.155829,A,C,M,B
4,0.380297,3733.04844,85.200147,14.103738,8.138688,3.942255,0.054810,3.396778,102.151980,5728.73412,...,16.408728,146.109943,8524.370502,45.381316,36.262628,0.096614,D,B,F,B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
612,0.149555,3130.05946,123.763599,9.513984,13.020852,3.499305,0.077343,8.545512,2.804172,4157.68439,...,8.967128,217.148554,8095.932828,24.640462,69.191944,21.978000,A,B,M,B
613,0.435846,5462.03438,85.200147,46.551007,15.973224,5.979825,0.025882,12.622906,3.777550,5654.07556,...,35.896418,496.994214,3085.308063,29.648928,124.808872,0.145340,A,B,M,B
614,0.427300,2459.10720,130.138587,55.355778,10.005552,8.070549,0.025578,15.408390,1.229900,5888.87769,...,19.962092,128.896894,6474.652866,26.166072,119.559420,21.978000,A,C,M,B
615,0.363205,1263.53524,85.200147,23.685856,8.138688,7.981959,0.025578,7.524588,1.229900,4517.86560,...,24.594488,72.611063,1965.343176,25.116750,37.155112,0.184622,A,C,M,B


In [4]:
def balanced_log_loss(y_true, y_pred):
    #https://www.kaggle.com/code/dan3dewey/icr-2023-balanced-log-loss
    N_0 = np.sum(1 - y_true)
    N_1 = np.sum(y_true)
    w_0 = 1 / N_0
    w_1 = 1 / N_1
    p_1 = np.clip(y_pred, 1e-15, 1 - 1e-15)
    p_0 = np.clip(1 - p_1, 1e-15, 1 - 1e-15) # had to clip again, numbers misbehaving
    log_loss_0 = -np.sum((1 - y_true) * np.log(p_0))
    log_loss_1 = -np.sum(y_true * np.log(p_1))
    balanced_log_loss = 2*(w_0 * log_loss_0 + w_1 * log_loss_1) / (w_0 + w_1)
    return balanced_log_loss/(N_0+N_1)
my_scorer = make_scorer(balanced_log_loss, needs_proba = True)

In [5]:
gp_pipe = Pipeline([("preprocessor", preprocessor),
                 ('oversampler', SMOTE()),
                 ('gp', ensemble.BaggingClassifier(estimator=gaussian_process.GaussianProcessClassifier(max_iter_predict = 1000),n_estimators=800))])
  
svc_pipe = Pipeline([("preprocessor", preprocessor),
                     ('svc', svm.SVC(kernel='rbf', max_iter = 10**9, C = 6, gamma = 0.7, probability = True, class_weight = 'balanced'))])

lr_pipe = Pipeline([("preprocessor", preprocessor),
                     ('oversampler', SMOTE()),
                     ('lr', ensemble.BaggingClassifier(estimator=linear_model.LogisticRegression(l1_ratio = 0.7, max_iter = 1000_000, solver = 'saga', penalty='elasticnet'),n_estimators=1000))])
  
kn_pipe = Pipeline([("preprocessor", preprocessor),
                 ('oversampler', SMOTE()),
                 ('kn', ensemble.BaggingClassifier(estimator=neighbors.KNeighborsClassifier(n_neighbors = 20, metric = 'manhattan', weights = 'distance', algorithm = 'kd_tree'),n_estimators=1000, max_samples = 0.7))])

ctb_pipe = Pipeline([("preprocessor", preprocessor),
                 ('oversampler', SMOTE()),
                 ('ctb', ctb.CatBoostClassifier(n_estimators = 1000, learning_rate = 0.01, verbose = False))])

tab_pipe = Pipeline([("preprocessor", preprocessor),
                 ('oversampler', SMOTE()),
                 ('tab', TabPFNClassifier())])

Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters


In [6]:
svc_weight = 1/10
lr_weight = 1/26
gp_weight = 1/28
knn_weight = 1/25
tab_weight = 1/30
ctb_weight = 1/18

## Ensemble

In [7]:
ensemble_model = ensemble.VotingClassifier(estimators=[('svc', svc_pipe), ('lr', lr_pipe), ('gp', gp_pipe), ('knn', kn_pipe),('tab_pipe', tab_pipe), ('ctb_pipe', ctb_pipe)],voting='soft', weights =[svc_weight, lr_weight, gp_weight, knn_weight, tab_weight, ctb_weight]) #0.207
#ensemble_score = cross_val_score(ensemble_model, train, classes, cv=3, scoring = my_scorer, n_jobs = -1)
#print(np.mean(ensemble_score), np.std(ensemble_score))

In [8]:
ensemble_model.fit(train, classes)

Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters


VotingClassifier(estimators=[('svc',
                              Pipeline(steps=[('preprocessor',
                                               ColumnTransformer(transformers=[('num',
                                                                                Pipeline(steps=[('imputer',
                                                                                                 SimpleImputer(missing_values=<NA>)),
                                                                                                ('scaler',
                                                                                                 MinMaxScaler())]),
                                                                                Index(['AB', 'AF', 'AH', 'AM', 'AR', 'AX', 'AY', 'AZ', 'BC', 'BD ', 'BN', 'BP',
       'BQ', 'BR', 'BZ', 'CB', 'CC', 'CD ', 'CF', 'CH', 'CL', 'CR', 'CS', 'CU',
       'CW ', 'DA', 'DE', 'DF', 'DH',...
                                                                                                 SimpleImputer(missing_values=<NA>,
                                                                                                               strategy='most_frequent')),
                                                                                                ('onehot',
                                                                                                 OneHotEncoder(handle_unknown='ignore'))]),
                                                                                ['EJ',
                                                                                 'Alpha',
                                                                                 'Beta',
                                                                                 'Gamma',
                                                                                 'Delta'])])),
                                              ('oversampler', SMOTE()),
                                              ('ctb',
                                               <catboost.core.CatBoostClassifier object at 0x7d372f3f5e40>)]))],
                 voting='soft',
                 weights=[0.1, 0.038461538461538464, 0.03571428571428571, 0.04,
                          0.03333333333333333, 0.05555555555555555])

In [9]:
#stacking_model = ensemble.StackingClassifier(estimators=[('svc', svc_pipe), ('lr', lr_pipe), ('gp', gp_pipe), ('knn', kn_pipe),('tab_pipe', tab_pipe), ('ctb_pipe', ctb_pipe)], final_estimator=linear_model.LogisticRegression())
#stacking_score = cross_val_score(stacking_model, train, classes, cv=3, scoring = my_scorer)
#print(np.mean(stacking_score), np.std(stacking_score))

In [10]:
#stacking_model.fit(train, classes)

## Submission

In [11]:
greeks_ensemble_model = load('/kaggle/input/icr-data-augment/greeks_augment_model.joblib')
test_data = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/test.csv')
test = test_data.drop(['Id'], axis=1)
predictions = np.array(greeks_ensemble_model.predict_proba(test))
class_columns = ['Alpha_A','Alpha_B','Alpha_D','Alpha_G','Beta_A','Beta_B','Beta_C','Gamma_A','Gamma_B','Gamma_E','Gamma_F','Gamma_G','Gamma_H','Gamma_M','Gamma_N','Delta_A','Delta_B','Delta_C','Delta_D']
predictions = pd.DataFrame(predictions[:, :, 1].T, columns= class_columns)
alphas = ['Alpha_A','Alpha_B','Alpha_D','Alpha_G']
betas = ['Beta_A','Beta_B','Beta_C']
gammas = ['Gamma_A','Gamma_B','Gamma_E','Gamma_F','Gamma_G','Gamma_H','Gamma_M','Gamma_N']
deltas = ['Delta_A','Delta_B','Delta_C','Delta_D']

def singularize(dataframe, columns):
    prediction = predictions[columns]
    arr = prediction.to_numpy()
    max_indices = np.argmax(arr, axis=1)
    result = np.zeros_like(arr)
    result[np.arange(result.shape[0]), max_indices] = 1
    return pd.DataFrame(result, columns = columns).astype('int32')

alphas = singularize(predictions, alphas)
betas = singularize(predictions, betas)
gammas = singularize(predictions, gammas)
deltas = singularize(predictions, deltas)

new_classes = alphas.join([betas, gammas, deltas])
categories = new_classes.columns.str.split('_').str[0].unique()
greeks = pd.DataFrame({category: new_classes[[col for col in new_classes.columns if category in col]].idxmax(axis=1).str.split('_').str[1] for category in categories})
display(greeks)

,Alpha,Beta,Gamma,Delta
0,A,C,M,B
1,A,C,M,B
2,A,C,M,B
3,A,C,M,B
4,A,C,M,B


In [12]:
test = pd.merge(test, greeks, left_index=True, right_index=True)
ensemble_proba = pd.DataFrame(ensemble_model.predict_proba(test), columns = ['class_0', 'class_1'])

submission = pd.DataFrame()
submission['Id'] = test_data['Id']
submission['class_0'] = ensemble_proba['class_0']
submission['class_1'] = ensemble_proba['class_1']
display(submission)

submission.to_csv('submission.csv', index=False)

,Id,class_0,class_1
0,00eed32682bb,0.736347,0.263653
1,010ebe33f668,0.736347,0.263653
2,02fa521e1838,0.736347,0.263653
3,040e15f562a2,0.736347,0.263653
4,046e85c7cc7f,0.736347,0.263653
